In [26]:
import pymc as pm
import numpy as np
import arviz as az
from pymc.math import log, exp

%load_ext lab_black
%load_ext watermark

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


# Deviance

This example demonstrates ...

Adapted from [unit 9: deviances.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit9/deviances.odc). The lecture has deviances2.odc, but that file wasn't included in the supporting material. We recreate it below anyways.

Todo:
- add more info about DIC and AIC
- add info about WAIC, link to papers showing advantages
    - https://arxiv.org/abs/1507.04544
    - https://arxiv.org/abs/1004.2316

## Associated lecture videos
### Unit 9 Lesson 1

In [2]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=88" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

### Unit 9 Lesson 2

In [3]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=89" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

In [4]:
t = np.array((1, 1, 2, 2, 3, 4, 4, 5, 5, 8))

## Model 1

In [22]:
with pm.Model() as m:
    gamma = pm.Gamma("gamma", 0.001, 0.001)
    mu = pm.Normal("mu", 0, tau=0.001)
    beta = mu ** (-1 / gamma)  # convert to PyMC parameterization

    f = pm.Deterministic(
        "f", mu * gamma * t ** (gamma - 1) * exp(-mu * t**gamma)
    )
    loglik = pm.Deterministic("loglik", log(f))
    pm.Deterministic("deviance", -2 * pm.math.sum(loglik))

    pm.Weibull("lik", gamma, beta, observed=t)

    trace = pm.sample(5000, tune=2000, target_accept=0.99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [gamma, mu]


/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 2_000 tune and 5_000 draw iterations (8_000 + 20_000 draws total) took 18 seconds.
There were 22 divergences after tuning. Increase `target_accept` or reparameterize.
There were 24 divergences after tuning. Increas

PyMC is not happy with this model, but the results agree with the professor's.

In [23]:
az.summary(trace, var_names=["mu", "gamma", "deviance"], kind="stats")

,mean,sd,hdi_3%,hdi_97%
mu,0.173,0.107,0.020,0.368
gamma,1.490,0.366,0.832,2.190
deviance,43.291,2.271,40.869,47.436


In [25]:
az.waic(trace, scale="deviance")

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/arviz/stats/stats.py:1661: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


Computed from 20000 posterior samples and 10 observations log-likelihood matrix.

              Estimate       SE
deviance_waic    44.84     4.68
p_waic            1.35        -

There has been a warning during the calculation. Please check the results.

## Model 2

In [27]:
with pm.Model() as m2:
    λ = pm.Gamma("λ", 0.001, 0.001)

    f = pm.Deterministic("f", λ * pm.math.exp(-λ * t))
    loglik = pm.Deterministic("loglik", log(f))
    pm.Deterministic("deviance", -2 * pm.math.sum(loglik))

    pm.Exponential("lik", λ, observed=t)

    trace = pm.sample(2000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [λ]


/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 10 seconds.


In [33]:
az.summary(trace, var_names=["λ", "deviance"], kind="stats")

,mean,sd,hdi_3%,hdi_97%
λ,0.288,0.090,0.120,0.454
deviance,46.053,1.432,45.055,48.648


In [37]:
az.waic(trace, scale="deviance")

Computed from 8000 posterior samples and 10 observations log-likelihood matrix.

              Estimate       SE
deviance_waic    46.49     3.85
p_waic            0.40        -

In [5]:
%watermark --iversions -v

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

pymc : 4.0.0
arviz: 0.12.1
numpy: 1.22.4

